# Tutorial 2: Using In-Disk Data in FastEstimator

In Tutorial 1, we introduced our 3 main APIs and general workflow of a deep learning task:  `Pipeline` -> `Network` -> `Estimator`. Then we used in-memory data for training. But what if the dataset size is too big to fit in memory? Say, data is in the size of ImageNet? 

The short answer is: user will use one more API for disk data: `RecordWriter`, such that the overall workflow becomes: `RecordWriter` -> `Pipeline` -> `Network` -> `Estimator`. In this tutorial, we are going to show you how to do in-disk data training in FastEstimator.

## Before we start:

Two things are required regarding in-disk data : 
* Data files, obviously :)
* A csv file that describes the data (prepare two csv files if you have a separate validation set)

In the csv file, the rows of csv represent different examples and columns represent different features within example. For example, for a classification task, a csv may look like:

| image  | label  |
|---|---|
|/data/image1.png   | 0  |
|/data/image2.png   |  1 |
|/data/image3.png | 0  |
|... | .  |

The csv of a multi-mask segmentation task may look like:

| img  | msk1  | msk2  |
|---|---|---|
|/data/image1.png   | /maska/mask1.png  |/maskb/mask1.png|
|/data/image2.png   |  /maska/mask2.png |/maskb/mask2.png|
|/data/image3.png | /maska/mask3.png  |/maskb/mask3.png|
|... | ...  |...|


Please keep in mind that, there is no restriction on the data folder structures, number of features or name of features. Now, let's generate some in-disk data for this tutorial:

In [ ]:
from fastestimator.dataset.mnist import load_data

train_csv, eval_csv, folder_path = load_data()
print("training csv path is {}".format(train_csv))
print("evaluation csv path is {}".format(eval_csv))
print("mnist image path is {}".format(folder_path))

Let's take a look at the csv file and image file

In [ ]:
import pandas as pd
df_train = pd.read_csv(train_csv)
df_train.head()

In [ ]:
import matplotlib.pyplot as plt
import os
img = plt.imread(os.path.join(folder_path, df_train["x"][1]))
plt.imshow(img)
print("ground truth of image is {}".format(df_train["y"][1]))

## Step 0: RecordWriter



In [ ]:
from fastestimator.record.preprocess import ImageReader
import fastestimator as fe
import tempfile

writer = fe.RecordWriter(save_dir=os.path.join(folder_path, "FEdata"),
                         train_data=train_csv,
                         validation_data=eval_csv,
                         ops=ImageReader(inputs="x", parent_path=folder_path, outputs="x"))

## Step 1 and above: Pipeline -> Network -> Estimator, same routine as tutorial 1:

In [ ]:
from fastestimator.pipeline.processing import Minmax
from fastestimator.architecture import LeNet
from fastestimator.network.model import FEModel
from fastestimator.network.model import ModelOp
from fastestimator.network.loss import SparseCategoricalCrossentropy

pipeline = fe.Pipeline(batch_size=32, data=writer, ops=Minmax(inputs="x", outputs="x"))

model = FEModel(model_def=LeNet, model_name="lenet", optimizer="adam", loss_name="loss")
network = fe.Network(ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), 
                          SparseCategoricalCrossentropy(y_pred="y_pred", y_true="y", outputs="loss")])
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=2)

## Start training

In [ ]:
estimator.fit()

In [ ]:
writer.ops_local